## Semantic Function Calling

In [1]:
#r "nuget:Microsoft.SemanticKernel"
#r "nuget: dotenv.net"
#r "nuget:Microsoft.SemanticKernel.Connectors.OpenAI"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureOpenAI"
#r "nuget:Aspose.Email"

Installed Packages Aspose.Email, 24.8.0 dotenv.net, 3.2.0 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.19.0 microsoft.semantickernel.connectors.openai, 1.19.0

In [62]:
using dotenv.net;

DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT");
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} ");
     

Using deployment: gpt-4o at https://scribeai.openai.azure.com/ 


# Setup Plugins

In [2]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using System.Text.Json.Serialization;

public class EmailPlugin
{
    [KernelFunction("send_email")]
    [Description("Sends an email to a recipient.")]
    public async Task SendEmailAsync(
        Kernel kernel,
        List<string> recipientEmails,
        string subject,
        string body
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Email Plugin to send emails!");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"Subject: {subject}");
        Console.WriteLine($"Body: {body}");
        
    }
}

public class EventModel
{
   [JsonPropertyName("person")]
   public string Person { get; set; }

   [JsonPropertyName("name")]
   public string Title { get; set; }

   [JsonPropertyName("date")]
   public DateTime Date { get; set; }

   [JsonPropertyName("notes")]
   public string Notes { get; set; }
   
}

public class CalendarPlugin
{
    [KernelFunction("get_calendar_events")]
    [Description("Gets calendar events for a given date.")]
    public async Task<List<EventModel>> GetCalendarAsync(
        Kernel kernel,
        List<string> recipientEmails,
        System.DateTime date
    )
    {
        // Add logic to get calendar events for recipientEmails on the given date
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Calendar Events Plugin started");
        Console.WriteLine($"Recipients: {string.Join(", ", recipientEmails)}");
        Console.WriteLine($"date: {date}");

        // Call Graph API to get calendar events !!!!!

        return   new()
            {
                new EventModel { Person = "Child1", Title = "Soccer practice", Date=DateTime.Parse("10/16/2024 12:00:00 AM"), Notes = "bring socks" },
                new EventModel { Person = "Child2", Title = "Dentist Appointment", Date=DateTime.Parse("10/15/2024 12:00:00 AM"), Notes = "bring healthcard, no food"},
                new EventModel { Person = "Adult2", Title = "Date night", Date=DateTime.Parse("10/15/2024 09:00:00 AM"), Notes = "Marche"}
            };
    }
}

In [64]:
using Aspose.Email;
using System;

public class EmailExtractionPlugin
{
    [KernelFunction("get_event_text")]
    [Description("Extract data from Email")]
    public async Task<string> ExtractEventFromEmail(
        Kernel kernel,
        string email_path
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("EmailAnalysis Plugin Started");

        MailMessage mail = MailMessage.Load("./data/" + "Event_Reminder_Development.eml");
        string body_without_url = mail.GetHtmlBodyText(false);// body will not contain URL
        // Console.WriteLine("Body with URL: " + body_with_url);
        // Console.WriteLine("Body without URL: " + body_without_url);  

        return body_without_url;

    }
}

# ASK Boss for Vacation

In [12]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;



// Create kernel with an email plugin
var builder = Kernel.CreateBuilder();
builder.Plugins.AddFromType<EmailPlugin>();
builder.Plugins.AddFromType<CalendarPlugin>();
builder.Plugins.AddFromType<EmailExtractionPlugin>();
Kernel kernel = builder.Build();

// Create chat completion service

AzureOpenAIChatCompletionService chatCompletionService = new (
    deploymentName: deploymentName,
    apiKey: apiKey,
    endpoint: endpoint
 );

// Enable planning
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

Console.WriteLine($" --------------------------------");
// Create chat history
var history = new ChatHistory();
history.AddUserMessage("Can you help me Elena write an email for my boss at boss@gmail.com subject: oh mine asking for vacation for days of Oct 15th and 16th in professional tone?");

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);

Console.WriteLine($" --------------------------------");
Console.WriteLine($" Email : {result}");
Console.WriteLine($" --------------------------------");



 --------------------------------
 --------------------------------
 Email : Here's a draft of your email:

---

Subject: Vacation Request for October 15th and 16th

Dear [Boss's Name],

I hope this email finds you well.

I am writing to formally request vacation days on October 15th and 16th. I have ensured that all my current projects are on track and will make sure to complete any pending tasks before my leave. During my absence, I will be reachable via email for any urgent matters.

Thank you for considering my request. Please let me know if you need any additional information.

Best regards,  
Elena

---

Would you like me to send this email on your behalf? If so, please confirm or provide any additional modifications you'd like to make.
 --------------------------------


# ASK Calendar events for vacation time

In [4]:
history.AddUserMessage("What are my events for vacation days,just list all of them, please?");

// Get the response from the AI
var vacresult = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);
Console.WriteLine($" --------------------------------");
Console.WriteLine($" Vacation events : {vacresult}");

Calendar Events
Recipients: boss@gmail.com
date: 10/15/2023 12:00:00 AM
Calendar Events
Recipients: boss@gmail.com
date: 10/16/2023 12:00:00 AM
 --------------------------------
 Vacation events : Here are your events for October 15th and 16th:

### October 15th:
1. **Person:** Child2
   - **Event:** Dentist Appointment
   - **Notes:** Bring health card, no food
2. **Person:** Adult2
   - **Event:** Date night
   - **Notes:** Marche

### October 16th:
1. **Person:** Child1
   - **Event:** Soccer practice
   - **Notes:** Bring socks

Would you like me to proceed with drafting the email to your boss, Elena?


In [68]:
history.Clear();
history.AddUserMessage("I have received email stored at './data/Event_Reminder_Development.eml' what event it descibes and do I have other events on that date?");

// Get the response from the AI
var evresult = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel
);
Console.WriteLine($" --------------------------------");
Console.WriteLine($" events : {evresult}");

EmailAnalysis Started
Calendar Events
Recipients: your_email@example.com
date: 10/19/2023 12:00:00 AM
 --------------------------------
 events : ### Event Described in the Email:

**Event Type:** Development Centre Reminder

- **Event Name:** U8-U10 DC (Wednesdays @ 6:30 PM)
- **Date:** Wednesday, September 11, 2024
- **Time:** 6:30 PM - 7:30 PM (Arrival Time: 6:25 PM)
- **Location:** NSG (Windsor Bubble), 808 Lytton St, North Vancouver, BC, Canada
- **Website:** [NSG Development Centre](https://www.nsg.com/development-centre/)
- **Label:** Fall Term
- **Notes:**
  - Bring players dressed in cleats, socks, shin guards, shorts/pants, t-shirt/sweater.
  - Players with long hair should have it tied up.
  - Players should bring a water bottle.
  - Players should leave jewelry at home.
  - Volunteers are needed to bring snacks, drinks, or help with other tasks.

### Other Events on that Date:
There are no events listed on your calendar for September 11, 2024.
